In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
#load df
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [7]:
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df=df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [8]:
import numpy as np
import pandas

In [58]:
#total votes >=20
vine_df_vote_filter=vine_df[vine_df.total_votes >= 20]
vine_df_vote_filter.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [59]:
vine_df_vote_filter2=vine_df_vote_filter.withColumn('helpful_vote_percentage', vine_df_vote_filter['helpful_votes']*100/vine_df_vote_filter['total_votes'])
vine_df_vote_filter2.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|                  100.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|                  100.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|                   80.0|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|                 96.875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|      73.84615384615384|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|      93.25842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|     

In [61]:
#helpful vote % >=50
vine_df_vote_filter3=vine_df_vote_filter2[vine_df_vote_filter2.helpful_vote_percentage >= 50]
vine_df_vote_filter3.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|                  100.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|                  100.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|                   80.0|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|                 96.875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|      73.84615384615384|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|      93.25842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|     

In [64]:
#paid reviews
vine_df_Y=vine_df_vote_filter3[vine_df_vote_filter3.vine == 'Y']
vine_df_Y.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|      87.09677419354838|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|      76.19047619047619|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|       87.8048780487805|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|      97.33333333333333|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|                  100.0|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|                  100.0|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|     

In [65]:
#unpaid reviews
vine_df_N=vine_df_vote_filter3[vine_df_vote_filter3.vine == 'N']
vine_df_N.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_vote_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|                  100.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|                  100.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|                   80.0|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|                 96.875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|      73.84615384615384|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|      93.25842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|     

In [66]:
#pandas paid df
pandas_paid_df = vine_df_Y.toPandas() 
pandas_paid_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_vote_percentage
0,R32OMS1LSDRCOB,4,27,31,Y,N,87.096774
1,R2IPKK4I0XRCFR,4,16,21,Y,N,76.190476
2,RXYYJ8SGI9IX6,5,36,41,Y,N,87.804878
3,R1SAUBFC7T6JHC,5,73,75,Y,N,97.333333
4,R329HVJKKHWSG3,5,59,59,Y,N,100.000000


In [68]:
#total reviews paid
y=pandas_paid_df.count()
y

review_id                  334
star_rating                334
helpful_votes              334
total_votes                334
vine                       334
verified_purchase          334
helpful_vote_percentage    334
dtype: int64

In [69]:
#5 star reviews paid
five_star_reviews_paid=pandas_paid_df[pandas_paid_df["star_rating"]=='5']
five_star_reviews_paid.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_vote_percentage
2,RXYYJ8SGI9IX6,5,36,41,Y,N,87.804878
3,R1SAUBFC7T6JHC,5,73,75,Y,N,97.333333
4,R329HVJKKHWSG3,5,59,59,Y,N,100.000000
5,RZL21K50M52KW,5,22,22,Y,N,100.000000
6,R24RC5J2D92SBE,5,19,20,Y,N,95.000000


In [71]:
#total reviews paid
x=five_star_reviews_paid.count()
x

review_id                  139
star_rating                139
helpful_votes              139
total_votes                139
vine                       139
verified_purchase          139
helpful_vote_percentage    139
dtype: int64

In [72]:
#percentage of 5 star reviews paid
percentage_paid=x/y*100
percentage_paid

review_id                  41.616766
star_rating                41.616766
helpful_votes              41.616766
total_votes                41.616766
vine                       41.616766
verified_purchase          41.616766
helpful_vote_percentage    41.616766
dtype: float64

In [73]:
#pandas unpaid df
pandas_unpaid_df = vine_df_N.toPandas() 
pandas_unpaid_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_vote_percentage
0,R1B4753QMTD8ON,5,32,32,N,N,100.000000
1,R28BVHOEL6GCHZ,5,21,21,N,Y,100.000000
2,R27IB4BRKO71I0,2,16,20,N,Y,80.000000
3,RJ86OW12Y4NRL,5,124,128,N,Y,96.875000
4,R3KRABOWCH6ZE4,1,48,65,N,Y,73.846154


In [74]:
#total reviews unpaid
y1=pandas_unpaid_df.count()
y1

review_id                  61614
star_rating                61614
helpful_votes              61614
total_votes                61614
vine                       61614
verified_purchase          61614
helpful_vote_percentage    61614
dtype: int64

In [75]:
#5 star reviews unpaid
five_star_reviews_unpaid=pandas_unpaid_df[pandas_unpaid_df["star_rating"]=='5']
five_star_reviews_unpaid.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,helpful_vote_percentage
0,R1B4753QMTD8ON,5,32,32,N,N,100.000000
1,R28BVHOEL6GCHZ,5,21,21,N,Y,100.000000
3,RJ86OW12Y4NRL,5,124,128,N,Y,96.875000
6,R36LSV4WQ4VB1K,5,17,22,N,N,77.272727
7,R18BTOE5UGN7UA,5,26,28,N,Y,92.857143


In [76]:
x1=five_star_reviews_unpaid.count()
x1

review_id                  32665
star_rating                32665
helpful_votes              32665
total_votes                32665
vine                       32665
verified_purchase          32665
helpful_vote_percentage    32665
dtype: int64

In [77]:
#percentage of 5 star reviews unpaid
percentage_unpaid=x1/y1*100
percentage_unpaid

review_id                  53.015548
star_rating                53.015548
helpful_votes              53.015548
total_votes                53.015548
vine                       53.015548
verified_purchase          53.015548
helpful_vote_percentage    53.015548
dtype: float64